In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
import numpy as np


In [2]:
# Load data 
df = pd.read_csv("../data/processed/final_training_data.csv", dtype={19: str, 20: str}, low_memory=False)

# Drop irrelevant columns
df = df.drop(columns=[
    'requestid', 'wktgeom', 'street', 'fromst', 'tost',
    'holiday_name', 'holiday_type', 'timestamp', 'weather_description'
])

# Add lag & rolling features
df = df.sort_values(['segmentid', 'year', 'month', 'day', 'hour', 'minute'])

df['vol_lag_1'] = df.groupby('segmentid')['vol'].shift(1)
df['vol_lag_2'] = df.groupby('segmentid')['vol'].shift(2)
df['vol_rolling_2'] = df.groupby('segmentid')['vol'].shift(1).rolling(2).mean().reset_index(level=0, drop=True)
df['vol_rolling_4'] = df.groupby('segmentid')['vol'].shift(1).rolling(4).mean().reset_index(level=0, drop=True)

# Drop rows with missing values from lag/rolling
df = df.dropna()

In [3]:
# Define target and features
y = df['vol']
X = df.drop(columns='vol')

# Column types
categorical_features = ['boro', 'direction', 'dayofweek']
numerical_features = [col for col in X.columns if col not in categorical_features]

In [4]:
# Define preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

# Define base pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(random_state=42, n_jobs=-1))
])

In [5]:
# Define hyperparameter grid (prefix with 'model__' to target the model step)
param_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [10, 20, None],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2],
    'model__max_features': ['sqrt', 'log2']
}

# Use TimeSeriesSplit for temporal validation
tscv = TimeSeriesSplit(n_splits=5)

In [9]:
# Wrap pipeline in GridSearchCV
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=tscv,
    scoring='neg_root_mean_squared_error',
    verbose=2,
    n_jobs=-1
)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn.preprocessing')

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# Fit grid search (this does training + validation internally)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__max_depth=10, model__max_features=sqrt, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=100; total time=   6.1s


/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__max_depth=10, model__max_features=sqrt, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=200; total time=  11.9s


...

/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__max_depth=10, model__max_features=log2, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=200; total time=  22.9s


/opt/anaconda3/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:246: UserWarning: Found unknown categories in columns [0] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


[CV] END model__max_depth=10, model__max_features=log2, model__min_samples_leaf=1, model__min_samples_split=5, model__n_estimators=100; total time=  12.2s


KeyboardInterrupt: 